In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import sys
from bokeh.plotting import figure, show, ColumnDataSource, save
from bokeh.models import Range1d, HoverTool
from bokeh.io import output_notebook, output_file
from gurobipy import *
output_notebook() 

Loading BokehJS ...

In [3]:
data = pd.read_csv('sample_data_daily.csv', index_col=0)
data.tail()

,PG,^GSPC,MSFT,AAPL,AC.TO,SU,BA,WMT,TD,ABT
Date,,,,,,,,,,
2020-03-23,97.699997,2237.399902,135.979996,224.369995,12.70,10.61,105.620003,114.279999,33.830002,62.820000
2020-03-24,103.269997,2447.330078,148.339996,246.880005,15.11,11.99,127.680000,115.029999,38.660000,69.690002
2020-03-25,100.919998,2475.560059,146.919998,245.520004,17.51,13.45,158.729996,109.400001,41.410000,70.750000
2020-03-26,107.379997,2630.070068,156.110001,258.440002,17.91,12.78,180.550003,109.820000,42.849998,75.809998
2020-03-27,110.169998,2541.469971,149.699997,247.740006,16.75,11.73,162.000000,109.580002,40.500000,74.559998


Get average return and volatility for each of the 10 stocks for the 24 month period

In [38]:
returns = data.pct_change()*100
stocks = returns.columns
covariance = returns.cov()
statistics = pd.concat((returns.mean(),returns.std()),axis=1)
statistics.columns = ['Avg_return', 'Volatility']
min_max = pd.concat((statistics.idxmin(),statistics.min(),statistics.idxmax(),statistics.max()),axis=1)
min_max.columns = ['Minimization','Minimum','Maximization','Maximum']
statistics

,Avg_return,Volatility
PG,0.095445,1.608335
^GSPC,0.008185,1.499359
MSFT,0.131456,2.025966
AAPL,0.108800,2.188743
AC.TO,-0.045274,2.922097
SU,-0.159394,2.509208
BA,-0.076587,3.196304
WMT,0.068711,1.488267
TD,-0.030479,1.729092
ABT,0.072131,1.785230


We are setting up the model. Add a variable for each stock, and set a lower-bound of 0 for each stock (weight)

In [68]:
model = Model("portfolio")
portfolio_variable = [model.addVar(name=symb,lb=0.0) for symb in stocks]
portfolio_variable = pd.Series(portfolio_variable, index=stocks)
pf = pd.DataFrame({'Variables':portfolio_variable})
model.update()

Constraint: want sum of weights to equal 1

In [69]:
model.addConstr(portfolio_variable.sum(), GRB.EQUAL, 1)

<gurobi.Constr *Awaiting Model Update*>

Obj fn: we want to minimize the objective function by minimizing the variance of the portfolio

In [70]:
pf_risk = covariance.dot(portfolio_variable).dot(portfolio_variable)
model.setObjective(pf_risk, GRB.MINIMIZE)
model.setParam('Method', 1)
model.optimize()

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x46345f8c
Model has 55 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 1 rows, 10 columns, 10 nonzeros
Presolved model has 55 quadratic objective terms

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s
       8    1.6665162e+00   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds
Optimal objective  1.666516183e+00


In [71]:
i = 0
weights = {}
for var in portfolio_variable:
    weights.update({stocks[i]:var.x})
    i = i + 1
weights = pd.DataFrame([weights])
weights = weights.transpose()
weights.columns = ['Weights']

print('\nStock, Optimal weight')
print(weights['Weights'])


Stock, Optimal weight
AAPL     0.000000
ABT      0.022230
AC.TO    0.060743
BA       0.000000
MSFT     0.000000
PG       0.180328
SU       0.002162
TD       0.161116
WMT      0.469448
^GSPC    0.103972
Name: Weights, dtype: float64


In [72]:
print('\nMin Portfolio Variance : '+str(pf_risk.getValue()))
min_volatility = math.sqrt(pf_risk.getValue())
print('Volatility : '+str(min_volatility))
avg_returns = returns.mean()
pf_return = avg_returns.dot(portfolio_variable)
exp_return = pf_return.getValue()
print('Expected Return : '+str(exp_return))


Min Portfolio Variance : 1.6665161828695019
Volatility : 1.2909361652961395
Expected Return : 0.043916744808138404


We have the minimum risk model. Below, we can see the minimum risk for each of the 10 stocks

In [73]:
pf['Min_risk'] = portfolio_variable.apply(lambda x:x.getAttr('x'))
pf

,Variables,Min_risk
PG,<gurobi.Var PG (value 0.18032783981262207)>,0.180328
^GSPC,<gurobi.Var ^GSPC (value 0.10397160068964069)>,0.103972
MSFT,<gurobi.Var MSFT (value 0.0)>,0.000000
AAPL,<gurobi.Var AAPL (value 0.0)>,0.000000
AC.TO,<gurobi.Var AC.TO (value 0.060743325024239105)>,0.060743
SU,<gurobi.Var SU (value 0.002162366392576545)>,0.002162
BA,<gurobi.Var BA (value 0.0)>,0.000000
WMT,<gurobi.Var WMT (value 0.46944820457942765)>,0.469448
TD,<gurobi.Var TD (value 0.16111620372116972)>,0.161116
ABT,<gurobi.Var ABT (value 0.02223045978032423)>,0.022230


In [74]:
portfolio_return = statistics['Avg_return'].dot(portfolio_variable)
portfolio_return

<gurobi.LinExpr: 0.09544505175015633 PG + 0.00818521259127366 ^GSPC + 0.13145560099468173 MSFT + 0.10879987804830572 AAPL + -0.04527368531424446 AC.TO + -0.15939389208766952 SU + -0.07658655029855738 BA + 0.06871085447602165 WMT + -0.03047877430155543 TD + 0.07213051300554632 ABT>

Finally, add a constraint to set the expected target return to be 50% of the maximum avg stock for one of the 10 stocks.

In [75]:
target_return = 0.5 * min_max.loc['Avg_return','Maximum']
target_return

0.06572780049734087

In [76]:
target = model.addConstr(portfolio_return, GRB.EQUAL, target_return)
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 2 rows, 10 columns and 20 nonzeros
Model has 55 quadratic objective terms
Coefficient statistics:
  Matrix range     [8e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-02, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6665162e+00   0.000000e+00   0.000000e+00      0s
       5    1.7202977e+00   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds
Optimal objective  1.720297704e+00


In [77]:
i = 0
weights = {}
for var in portfolio_variable:
    weights.update({stocks[i]:var.x})
    i = i + 1
weights = pd.DataFrame([weights])
weights = weights.transpose()
weights.columns = ['Weights']

print('\nStock, Optimal weight')
print(weights['Weights'])


Stock, Optimal weight
AAPL     0.005869
ABT      0.123095
AC.TO    0.060279
BA       0.000000
MSFT     0.000000
PG       0.289116
SU       0.000000
TD       0.045344
WMT      0.476296
^GSPC    0.000000
Name: Weights, dtype: float64


Now lets make the efficient frontier (pareto optimal curve). We will go through different retuns and make a minimum-risk portfolio for each.

In [78]:
model.setParam('OutputFlag',False)
min_return = min_max.loc['Avg_return','Minimum']
max_return = min_max.loc['Avg_return','Maximum']
risk_return = min_max.loc['Volatility','Minimization']
risk_return = statistics.loc[risk_return,'Avg_return']
risk_return =sum(pf['Min_risk']*statistics['Avg_return'])
return_range = np.unique(np.hstack((np.linspace(min_return,max_return,10000),risk_return)))

In [79]:
stock_risks = return_range.copy()
for i in range(len(return_range)):
    target.rhs = return_range[i]
    model.optimize()
    stock_risks[i] = sqrt((covariance.dot(portfolio_variable).dot(portfolio_variable)).getValue())

Separate curve into 2 sections: red for those stocks with a return less than the minimum risk portfolio; and blue for those stocks with a greater return

In [80]:
efficient_frontier = figure(tools="pan,box_zoom,reset")

efficient_frontier.circle(statistics['Volatility'], statistics['Avg_return'])
efficient_frontier.text(statistics['Volatility'], statistics['Avg_return'], stocks)

efficient_frontier.line(stock_risks[return_range <= risk_return], return_range[return_range <= risk_return], color='red')
efficient_frontier.line(stock_risks[return_range >= risk_return], return_range[return_range >= risk_return], color='blue')

efficient_frontier.xaxis.axis_label='Volatility'
efficient_frontier.yaxis.axis_label='Average Return'
show(efficient_frontier)